In [1]:
import cv2
import numpy as np
import pandas as pd
import os
import random
import sys
sys.path.insert(0, './lib/')

from lib.help_functions import dir_checker


In [2]:
def get_raw_data_frame(target_video, save_folder,sampling_time = 1, each_folder = True, average_mode = False):
    if os.path.isdir('./'+save_folder) == False:
        os.mkdir('./'+save_folder)
    else:
        print('already exist the folder in this path : {}'.format('./'+save_folder)) 
    
    video_cap = cv2.VideoCapture(target_video)
    fps = int(video_cap.get(5))

    success, imgs = video_cap.read()
    count = 0 
    sec = 0
    
    print('fps : ',fps)
    
    while success:
        if each_folder ==True:
            if os.path.isdir('./'+save_folder+str(sec)+'/') == True:
                pass
            else:
                os.mkdir('./'+save_folder+str(sec)+'/')
            cv2.imwrite(save_folder+str(sec)+'/'+str(count)+'.png',imgs)
            success, imgs = video_cap.read()
            count = count +1
            if count == fps:
                sec = sec+1
                count = 0
        else:
            cv2.imwrite(save_folder+str(count)+'.png',imgs)
            success, imgs = video_cap.read()
            count = count +1
            

def resize_video(target_video, save_folder,resize_shape, average_mode = False, avg_img_path = False):
    if os.path.isdir(save_folder) == False:
        os.mkdir(save_folder)
    else:
        print('already exist the folder in this path : {}'.format('./'+save_folder)) 
    
    video_cap = cv2.VideoCapture(target_video)
    fps = int(video_cap.get(5))

    success, imgs = video_cap.read()
    #print('img shape : ', np.shape(imgs))
    count = 0 
    sec = 0
    
    imgs_array = []
    padded_img = np.zeros((resize_shape[0], resize_shape[1], resize_shape[2]))
    print('[debug] padded img : ', np.shape(padded_img))
    
    
    print('fps : ',fps)

    img_h, img_w = np.shape(imgs)[0], np.shape(imgs)[1]
    padded_img[:img_h, :img_w] = imgs
    print('img h : {} img w : {} '.format(img_h, img_w))
    
    while success:
        imgs_array.append(padded_img)
        if average_mode == True:
            avg_img = cv2.imread(avg_img_path)
            print('avg shape : ', np.shape(avg_img))
            avg_img_h, avg_img_w = np.shape(avg_img)[0], np.shape(avg_img)[1]
            padded_avg_img = np.zeros((resize_shape[0], resize_shape[1], resize_shape[2]))
            padded_avg_img[:avg_img_h, :avg_img_w] = avg_img
            cv2.imwrite(save_folder+'0'+str(count)+'.png', padded_avg_img)
            
            count = count+1
            cv2.imwrite(save_folder+'0'+str(count)+'.png',padded_img)
            
            success, imgs = video_cap.read()
            padded_img[:img_h, :img_w] = imgs
            count = count+1
            average_mode = False
            pass
        
        if 10 > count:
            cv2.imwrite(save_folder+'0'+str(count)+'.png',padded_img)
        else:
            cv2.imwrite(save_folder+str(count)+'.png',padded_img)
        
        
        success, imgs = video_cap.read()
        padded_img[:img_h, :img_w] = imgs
        count = count +1
    print('Done')
            
            

In [3]:
!pwd

/home/bono/Desktop/mellab_project/bono_hw/unet_retinal


In [4]:
resize_shape = (1848, 2208, 3)
target_video01 = './data/201130/data/15.P.Y.M Registered.avi'
target_avg_path = './data/201130/data/15.P.Y.M Registered_avg.png'

# /home/bono/Desktop/mellab_project/bono_hw/unet_retinal
save_folder01 = './data/201130/proc_data/'
dir_checker(save_folder01)



resize_video(target_video01,save_folder01,resize_shape, average_mode=True, avg_img_path=target_avg_path)


already exist the folder in this path : ./data/201130/proc_data/
already exist the folder in this path : ././data/201130/proc_data/
[debug] padded img :  (1848, 2208, 3)
fps :  26
img h : 1848 img w : 2208 
avg shape :  (1848, 2208, 3)
Done


In [ ]:
### def non_uniform_back(target_img, blur_size,debug_option = 'off'):
    """non uniform background subtraction.

    Parameters
    ----------
    image : (M, N[, C]) 2D numpy array / (pixel / color space)
        2D numpy array image
    
    blur_size : median blur kernel size
        recommended kernel size is 30~40.
        it it up to your image.
      
    debug_option : for debugging.
        debugging option == 'on'
            show histogram & show data tape
            (default = 'off')
            
    Returns
    -------
        uniform background image 't'
    
    Example
    -------

    """
    img = target_img.copy()
    
    for img_num in range(img.shape[0]):
        nUniImg = img[img_num,0]
        backImg = median(nUniImg,disk(51))
        meanVal = np.mean(nUniImg)
        
        img[img_num,0] = np.divide(nUniImg,backImg) * meanVal
        col,row = nUniImg.shape[0],nUniImg.shape[1]
        for i in range(col):
            for j in range(row):
                if img[img_num,0][i,j] <= 0:
                    img[img_num,0][i,j] = 0
                elif img[img_num,0][i,j] >=255:
                    img[img_num,0][i,j] = 255
                    
        if debug_option == 'on':
            print('data type \n')
            print('Img : {}, backImg : {}, resultImg : {}'.format(nUniImg.dtype,backImg.dtype,t.dtype))

            plt.hist(shadeResult.ravel(),256,[shadeResult.min(),shadeResult.max()])
            plt.title('Histogram')
            plt.show()
            
    return img

def correction_non_uniform_back(img, blur_size,debug_option = 'off'):
    """non uniform background subtraction.
    by this paper: 
    Automated segmentation of the optic nerve head for diagnosis of glaucoma
    
    R.Chrastek, M.Wolf, K.Donath, H.Niemann, D.Paulus, T. Hothorn, B.Lausen, R.Lammer, C.Y.Mardin, G.Michelson
    
    MEDICAL IMAGE ANALYSIS
    
    correction of non-uniform illuminaition.

    Parameters
    ----------
    image : (M, N[, C]) 2D numpy array / (pixel / color space)
        2D numpy array image
    
    blur_size: median blur kernel size
        recommended kernel size is 30~40.
        it it up to your image.
    
    debug_option : for debugging.
        debugging option == 'on'
            show histogram & show data tape
            (default = 'off')
            
    Returns
    -------
        uniform background image 't'
    
    Example
    -------

    """
    if img.shape[2] == None:
        corImg = img
        backImg = cv2.medianBlur(corImg,blur_size)
        maxGrayVal = np.max(backImg)
        
        col,row = corImg.shape[0], corImg.shape[1]
        rList = np.zeros((col,row))

        for i in range(col):
            for j in range(row):
                if backImg[i,j] != 0:
                    rList[i,j] = maxGrayVal / backImg[i,j]
                    
        meanVal =np.mean(backImg,dtype = 'int') 
        c = maxGrayVal - meanVal
        p = np.multiply(img,rList)
        t = p - c
        for i in range(col):
            for j in range(row):
                if t[i][j] >= 255:
                    t[i][j] = 255
                elif t[i][j] <=0:
                    t[i][j] = 0
        
        if debug_option == 'on':
            print('data type \n')
            print('corImg : {}, backImg : {}, resultImg : {}'.format(corImg.dtype,backImg.dtype,t.dtype))
            
            plt.hist(t.ravel(),256,[t.min(),t.max()])
            plt.title('Histogram')
            plt.show()
            
        return t
        
    elif img.shape[2] == 3:
        corImg = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        backImg = cv2.medianBlur(corImg,blur_size)
        maxGrayVal = np.max(backImg)
        
        col,row = corImg.shape[0], corImg.shape[1]
        rList = np.zeros((col,row))

        for i in range(col):
            for j in range(row):
                if backImg[i,j] != 0:
                    rList[i,j] = maxGrayVal / backImg[i,j]
                    
        meanVal =np.mean(backImg,dtype = 'int') 
        c = maxGrayVal - meanVal
        p = np.multiply(img,rList)
        t = p - c
        for i in range(col):
            for j in range(row):
                if t[i][j] >= 255:
                    t[i][j] = 255
                elif t[i][j] <=0:
                    t[i][j] = 0
        if debug_option == 'on':
            print('data type \n')
            print('corImg : {}, backImg : {}, resultImg : {}'.format(corImg.dtype,backImg.dtype,t.dtype))

            plt.hist(t.ravel(),256,[t.min(),t.max()])
            plt.title('Histogram')
            plt.show()            
        return t
    else:
        print('Color Space Error!')

In [10]:
target_video_path = './data/12.26/6_crop.avi'
save_folder_path = './data/12.26/frame_data_6_crop/'


#for i in range(11):
get_raw_data_frame(target_video_path, save_folder_path, each_folder=False)

fps :  21


In [11]:
temp = cv2.imread('./data/12.26/frame_data_6_crop/0.png')
print(np.shape(temp))

(1000, 1000, 3)


In [ ]:
def make_clip(file_path, clip_duration=20, clip_fps=15):
    # config
    dir_path_, ext_ = os.path.splitext(file_path)

    # for insert and save video files
    parents_path, _ = os.path.split(file_path)

    # for saving dirctory
    new_dir_path = parents_path + '/_clips'

    clip_frames_ = clip_duration * clip_fps

    # clips config
    fourcc = cv2.VideoWriter_fourcc(*'H264')
    fps_ = 15
    size_ = (1920, 1080)

    # make directory to saving video clips
    if os.path.isdir(new_dir_path):
        pass
    else:
        os.mkdir(new_dir_path)

    # read video and compute number of clips
    cap = cv2.VideoCapture(file_path)

    frame_nums_ = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    clip_nums_ = math.ceil(frame_nums_ / clip_frames_)

    # for recursion
    # fc = 0

    while cap.isOpened():

        ret, frame = cap.read()

        # make clips
        for clip_ in range(clip_nums_):

            clip_path_ = new_dir_path + '/clip_{}'.format(clip_) + ext_
            out = cv2.VideoWriter(clip_path_, fourcc, fps_, size_)

            if ret:
                for i in range(clip_frames_):
                    out.write(frame)
                    # fc += 1
            else:
                # fc += 1
                out.release()
    cap.release()

    return print('success to make clips and directory.')